# ASSIGNMENT

In [15]:
# LINK - https://drive.google.com/drive/folders/1GY_-vRTkAdx7i7Sp4ZbCfO2yiG-iSNRJ
# I used the 50 MB dataset

"""POTATO Take-Home Task
Overview:
POTATO (the Panel-based Open Term-level Aggregate Twitter Observatory) is a prototype website that uses data 
from the Lazer Lab’s Twitter Panel. The Twitter Panel links over one million real U.S. voters to their Twitter 
accounts, and has each panelist’s tweets from about 2016 until 2023 or so. POTATO will allow users to search 
for a term (“COVID”) and get aggregate information about the people who tweeted about the term. We will 
threshold results such that any demographic bucket with fewer than ten users will not be shown; we’re also 
considering using statistical processes to add noise without disrupting the overall distribution of the data. 
Right now the system uses Docker, Elasticsearch, Streamlit, and Python. Our biggest technical problems are a) 
ingesting the data from HDFS efficiently and b) returning results quickly. We also need to look into 
strengthening our privacy protections."""

'POTATO Take-Home Task\nOverview:\nPOTATO (the Panel-based Open Term-level Aggregate Twitter Observatory) is a prototype website that uses data \nfrom the Lazer Lab’s Twitter Panel. The Twitter Panel links over one million real U.S. voters to their Twitter \naccounts, and has each panelist’s tweets from about 2016 until 2023 or so. POTATO will allow users to search \nfor a term (“COVID”) and get aggregate information about the people who tweeted about the term. We will \nthreshold results such that any demographic bucket with fewer than ten users will not be shown; we’re also \nconsidering using statistical processes to add noise without disrupting the overall distribution of the data. \nRight now the system uses Docker, Elasticsearch, Streamlit, and Python. Our biggest technical problems are a) \ningesting the data from HDFS efficiently and b) returning results quickly. We also need to look into \nstrengthening our privacy protections.'

# Part 1

# Ingest the data: figure out a way to put the data in a structure so that you can query it as described in Part 2.

In [16]:
# Importing necessary libraries
import warnings as warn
warn.filterwarnings("ignore")
import pandas as pd 

# We will load the tweet data into a Pandas DataFrame. 
twitter_ = pd.read_csv("C:\Python programming\correct_twitter_201904.tsv", sep="\t")
twitter_.head()

,id,event,ts1,ts2,from_stream,directly_from_stream,from_search,directly_from_search,from_quote_search,directly_from_quote_search,...,retweeted,retweeted_author_id,retweeted_handle,retweeted_follower_count,mentioned_author_ids,mentioned_handles,hashtags,urls,media_keys,place_id
0,1131594960443199488,britney_201904,2022-02-28 09:34:44.627023-05:00,2022-02-28 09:34:44.627023-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1131594976750653440,britney_201904,2022-02-28 09:34:44.626921-05:00,2022-02-28 09:34:44.626921-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1131589737955942405,britney_201904,2022-02-28 09:34:44.634058-05:00,2022-02-28 09:34:44.634058-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1131594909469892610,britney_201904,2022-02-28 09:34:44.627125-05:00,2022-02-28 09:34:44.627125-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1131594812694511617,britney_201904,2022-02-28 09:34:44.627227-05:00,2022-02-28 09:34:44.627227-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# drop_duplicated row
# Checking the ID column whether it contain any duplicate row or not
print("'id' column is unique, no duplicate present") if twitter_.duplicated(subset="id", keep="first").sum() == 0 else \
    print("ID column is not unique")

'id' column is unique, no duplicate present


In [18]:
# Checking the information
twitter_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88037 entries, 0 to 88036
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             88037 non-null  int64  
 1   event                          88037 non-null  object 
 2   ts1                            88037 non-null  object 
 3    ts2                           88037 non-null  object 
 4   from_stream                    88037 non-null  bool   
 5   directly_from_stream           88037 non-null  bool   
 6   from_search                    88037 non-null  bool   
 7   directly_from_search           88037 non-null  bool   
 8   from_quote_search              88037 non-null  bool   
 9   directly_from_quote_search     88037 non-null  bool   
 10  from_convo_search              88037 non-null  bool   
 11  directly_from_convo_search     88037 non-null  bool   
 12  from_timeline_search           88037 non-null 

In [19]:
# Columns aligment
twitter_.columns = twitter_.columns.str.strip()

twitter_.columns

Index(['id', 'event', 'ts1', 'ts2', 'from_stream', 'directly_from_stream',
       'from_search', 'directly_from_search', 'from_quote_search',
       'directly_from_quote_search', 'from_convo_search',
       'directly_from_convo_search', 'from_timeline_search',
       'directly_from_timeline_search', 'text', 'lang', 'author_id',
       'author_handle', 'created_at', 'conversation_id', 'possibly_sensitive',
       'reply_settings', 'source', 'author_follower_count', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'replied_to',
       'replied_to_author_id', 'replied_to_handle',
       'replied_to_follower_count', 'quoted', 'quoted_author_id',
       'quoted_handle', 'quoted_follower_count', 'retweeted',
       'retweeted_author_id', 'retweeted_handle', 'retweeted_follower_count',
       'mentioned_author_ids', 'mentioned_handles', 'hashtags', 'urls',
       'media_keys', 'place_id'],
      dtype='object')

In [20]:
# Converting the 'ts1', 'ts2' into timestamp
twitter_["ts1"] = pd.to_datetime(twitter_["ts1"])
twitter_["ts2"] = pd.to_datetime(twitter_["ts2"])

# Part 2

# Construct functionality that allows you to query the data. If I search for a term, like “music,” I would like to know some subset of the following:
#### - How many tweets were posted containing the term on each day?
#### - How many unique users posted a tweet containing the term?
#### - How many likes did tweets containing the term get, on average?
#### - Where (in terms of place IDs) did the tweets come from?
#### - What times of day were the tweets posted at? 
#### - Which user posted the most tweets containing the term?


In [21]:
class Twitter_Tweets:
    
    def __init__(self, term):
        self.term = term

    def count_tweets_by_day(self):
        """
        Count the number of tweets containing a term on each day.

        Parameters:
        term (str): The term to search for in tweets.

        Returns:
        DataFrame: A DataFrame with dates and tweet counts.
        """
        # Filter for tweets containing the term
        filtered_df1 = twitter_[twitter_['text'].str.contains(self.term, case=False, na=False)]

        # Extract the date from the timestamp
        filtered_df1['date'] = filtered_df1['ts1'].dt.date

        # Group by date and count
        tweet_counts = filtered_df1.groupby('date').size().reset_index(name='tweet_count')

        return tweet_counts
    
    def count_unique_users_by_term(self):
        # Filter for tweets containing the term
        filtered_df = twitter_[
            twitter_['text'].str.contains(self.term, case=False, na=False)
        ]

        # Count unique users
        unique_count = filtered_df['author_id'].nunique()

        return unique_count
    
    def average_likes_by_term(self):
        # Filter for tweets containing the term
        filtered_df03 = twitter_[
            twitter_['text'].str.contains(self.term, case=False, na=False)
        ]

        # Calculate the average likes
        if filtered_df03.empty:
            avg_likes = 0
        else:
            avg_likes = filtered_df03['like_count'].mean()

        return avg_likes
    
    def places_of_tweets_by_term(self):
        # Filter for tweets containing the term
        filtered_df = twitter_[
            twitter_['text'].str.contains(self.term, case=False, na=False)
        ]

        # Get the unique place IDs
        unique_place_ids = filtered_df['place_id'].dropna().unique()  # Drop NaN values

        return unique_place_ids
    
    def times_of_day_by_term(self):
        # Filter for tweets containing the term "music"
        filtered_df = twitter_[twitter_['text'].str.contains(self.term, case=False, na=False)]

        # Extract the time from the timestamp
        filtered_df['time'] = filtered_df['ts1'].dt.time

        ## value count and convert it as dataframe
        time_counts = filtered_df['time'].value_counts().reset_index(name='tweet_count')

        ## changing the columns name
        time_counts.columns = ['time', 'tweet_count']

        return time_counts
    
    def most_active_user_by_term(self):
        # Filter for tweets containing the term "music"
        filtered_df = twitter_[twitter_['text'].str.contains(self.term, case=False, na=False)]

        # Checking for Tweets:
        if filtered_df.empty:
            return 0  # No tweets found
        
        # Counting Author IDs : 
        """returns the author_id that has the highest count, which corresponds to the most active user."""
        most_active_user = filtered_df['author_id'].value_counts().idxmax()
        tweet_count = filtered_df['author_id'].value_counts().max()
        return most_active_user, tweet_count


    

print("term is 'music'")
term = input("Enter the tweet term : ")

print("question1 : How many tweets were posted containing the term on each day?")
# question 1
## How many tweets were posted containing the term on each day?
result1 = Twitter_Tweets(term=term)
print(result1.count_tweets_by_day())

print()

print("question2 : How many unique users posted a tweet containing the term?")
# question 2 
## How many unique users posted a tweet containing the term?
result2 = Twitter_Tweets(term=term)
print(result2.count_unique_users_by_term())

print()

print("question3 : How many likes did tweets containing the term get, on average?")
result3 = Twitter_Tweets(term=term)
print(result3.average_likes_by_term())

print()

print("question4 : Where (in terms of place IDs) did the tweets come from?")
result4 = Twitter_Tweets(term=term)
print(result4.places_of_tweets_by_term())

print()

print("question5 : What times of day were the tweets posted at? ")
result5 = Twitter_Tweets(term=term)
print(result5.times_of_day_by_term().head())


print()

print("Question6 : Which user posted the most tweets containing the term?")
result6 = Twitter_Tweets(term=term)
print(result6.most_active_user_by_term())

term is 'music'
question1 : How many tweets were posted containing the term on each day?
         date  tweet_count
0  2022-01-04           66
1  2022-02-28         2935

question2 : How many unique users posted a tweet containing the term?
2109

question3 : How many likes did tweets containing the term get, on average?
161.40853048983672

question4 : Where (in terms of place IDs) did the tweets come from?
['53504716d445dcad' 'ab2f2fac83aa388d' '0113afc024d5e0bc'
 '300bcc6e23a88361' '8e9665cec9370f0f' 'd56c5babcffde8ef'
 '01153d1b33e1641b' '09f6a7707f18e0b1' '714789cf3b7a50d0'
 '01fbe706f872cb32' '1a7a70d4a28e96a1' 'f97108ab3c4a42ed'
 'ac88a4f17a51c7fc' '5de8cffc145c486b' '3f7a925ec706ea48'
 '001aff55522d96c9' '4c8e28554110ebcf' '5c62ffb0f0f3479d'
 '3b77caf94bfc81fe' 'ecbe2aea853af44e' 'de599025180e2ee7'
 '00b673715a35dfa7' '43d2418301bf1a49' '07e9c7d1954fff64'
 '0570f015c264cbd9' '0149775319466b18' '91890dbb74364d63'
 '01ddb0100b1efd97' 'b49b3053b5c25bf5' '7cb7440bcf83d464'
 '151b9e91